# Example of powering an interactive dashboard via Redshift

In [7]:
# Imports
import dash
import dash_core_components as dcc
import dash_html_components as html

import calendar
import pandas as pd
import numpy as np

In [8]:
# Build AppViewer 
from jupyterlab_dash import AppViewer
viewer = AppViewer()

In [9]:
# Local test - set up connection
%run ./db_common_local.py

conn = default_conn

In [ ]:
# Sagemaker notebook env
%run ./db_common.py

conn = default_conn

## Visual for TPCDS Query 21
For all items whose price was changed on a given date, compute the percentage change in inventory 
between the 30-day period BEFORE the price change and the 30-day period AFTER the change.


In [10]:
query_stmt = '''
select  *
 from(select w_warehouse_name
            ,i_item_id
            ,i_current_price
            ,sum(case when (cast(d_date as date) < cast ('2001-%s-11' as date))
                  then inv_quantity_on_hand 
                      else 0 end) as inv_before
            ,sum(case when (cast(d_date as date) >= cast ('2001-%s-11' as date))
                      then inv_quantity_on_hand 
                      else 0 end) as inv_after
   from inventory
       ,warehouse
       ,item
       ,date_dim
   where i_current_price between 0.50 and 1.49
     and i_item_sk          = inv_item_sk
      %s
     and inv_warehouse_sk   = w_warehouse_sk
     and inv_date_sk    = d_date_sk
     and d_date between dateadd(day, -30, '2001-%s-11') 
                    and dateadd(day, 30, '2001-%s-11')
   group by w_warehouse_name, i_item_id,i_current_price) x
 where (case when inv_before > 0 
             then inv_after / inv_before 
             else null
             end) between 2.0/3.0 and 3.0/2.0
 order by w_warehouse_name
         ,i_item_id        
 limit 100;
'''


In [11]:
# Initialize
dataset = 'tpcds_100gb'

search_path = 'set search_path to %s;' % dataset
exec_dml(conn, search_path)

df = exec_query(conn, 'select distinct(w_warehouse_name) from warehouse where len(w_warehouse_name) > 0 order by 1;')
available_items = df['w_warehouse_name'].unique()
available_items = np.insert(available_items, 0, 'ALL')

initial_month = '4'
initial_item = 'Conventional childr'


Escaped time: 0 seconds
Escaped time: 0 seconds, 	Returned rows: 14


## Interactive dashboard
- Select a different warehouse
- Or slide to choose another month

In [12]:

app = dash.Dash("Redshift dashboard")

app.layout = html.Div([
    html.Div([
        html.Div([
            html.Div(children='Warehouses: '),
            dcc.Dropdown(
                    id='input_item',
                    options=[{'label': i, 'value': i} for i in available_items],
                    value='ALL'
                )
        ],
        style={'width': '40%', 'display': 'inline-block'}),
        html.Div([
            html.Div(children='Slide to choose another month: '),
            dcc.Slider(id='input_month', value=int(initial_month), min=1, max=12, step=1,
                   marks={
                            1: {'label': 'Jan'},
                            2: {'label': 'Feb'},
                            3: {'label': 'Mar'},
                            4: {'label': 'Apr'},
                            5: {'label': 'May'},
                            6: {'label': 'Jun'},
                            7: {'label': 'Jul'},
                            8: {'label': 'Aug'},
                            9: {'label': 'Sept'},
                            10: {'label': 'Oct'},
                            11: {'label': 'Nov'},
                            12: {'label': 'Dec'}
                        }
                )
        ], style={'width': '80%', 'display': 'inline-block'})
    ]),
    html.Div(children='_ '),

    html.Div([
        dcc.Graph(id='output-graph')
    ], style={'width': '80%', 'display': 'inline-block'})    
])

@app.callback(
    dash.dependencies.Output('output-graph', 'figure'),
    [dash.dependencies.Input('input_month', 'value'),
     dash.dependencies.Input('input_item', 'value')]
)

def update_graph(input_month, input_item):
    if input_item == 'ALL':
        input_item = ''
    else:
        input_item = ''' and w_warehouse_name = '%s' ''' % input_item
    query = query_stmt % (input_month,input_month,input_item,input_month,input_month)
    query = query.replace('\n', ' ').replace('\r', '')
    df_result = exec_query(conn, query)

    return {
            'data' : [
                {'x' : df_result.i_item_id, 'y' : df_result.inv_before, 'type' : 'bar', 'name' : 'inventory before'},
                {'x' : df_result.i_item_id, 'y' : df_result.inv_after, 'type' : 'line', 'name' : 'inventory after'},
            ],
            'layout' : {
                'title' : 'Inventory change in %s' % calendar.month_name[input_month]
            }
        }

viewer.show(app)